<a href="https://colab.research.google.com/github/deepakrana123us/Bike-Sharing-Assignment/blob/main/Copy_of_Travel_Planner_Deepak_Rana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Description:**
Objective: Travel Planner AI: Develop a chatbot that assists travellers in planning their trips. It can recommend top hotels and suggest itineraries based on user-specified destinations.This project leverages advanced AI technologies to create a seamless and personalized travel experience. Key features include:
Travel Places Recommendation: Using a recommendation model, the system suggests travel destinations based on user preferences and historical data, offering tailored travel options for every user.



In [1]:
!pip install fuzzywuzzy


In [2]:
## Import all the libraries
import google.generativeai as genai
import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations, permutations
import ast
import time
import dill
from httpx import HTTPStatusError
import tempfile
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz # for string matching
from fuzzywuzzy import process # for string matching
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm  # Import tqdm for progress logging
import logging
from textblob import TextBlob
import google.generativeai as genai
from collections import defaultdict
import time

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:

from google.colab import userdata
key_1 = userdata.get('GEMINI_API_KEY')


In [4]:
#Setup for Travel planner
genai.configure(api_key=key_1)

generation_config1 = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 20240,
    "response_mime_type": "text/plain",
}

model1 = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config1,
)

In [5]:
# Mount google drive because the input data is stored there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
places_df = pd.read_csv('/content/drive/MyDrive/Places Dataset.csv')
users_df = pd.read_csv('/content/drive/MyDrive/Visitors Preference.csv')

In [ ]:
places_df.head()

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
0,Arugam Bay Beach,6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,Mirissa Beach,5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri LankaÃ¢Â...
2,Weligama Beach (surf and stay),5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
3,Ahangama,5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...
4,Hikkaduwa Beach,6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...


In [7]:
users_df.head()

,User ID,Name,Email,Preferred Activities,Bucket list destinations Sri Lanka
0,1,Jennifer Quinn,jennifer.quinn@example.com,"['cycling', 'historical monuments', 'village h...","['Polonnaruwa', 'Hatton', 'Anuradhapura', 'Ell..."
1,2,Emily Perry,emily.perry@example.com,"['butterfly watching', 'hot springs', 'wildlif...","['Madunagala Hot Water Spring', 'Wilpattu Nati..."
2,3,Danielle Mcbride,danielle.mcbride@example.com,"['sea cruises', 'themed parks', 'craft worksho...","['Mirissa Beach', 'Negombo Lagoon', 'Batadomba..."
3,4,Angelica Wilson,angelica.wilson@example.com,"['fishing', 'hot springs', 'sailing']","['Maha Oya Hot Water Springs', 'Colombo Port C..."
4,5,Laurie Powers,laurie.powers@example.com,"['history tours', 'sailing', 'literary tours']","['Negombo Lagoon', 'Colombo Port City', 'Galle..."


In [8]:
#Function to analyze data
def customDescription(df: pd.DataFrame, numeric_only: bool = False):
    if numeric_only:
        df = df.select_dtypes(include=np.number)

    desc = pd.DataFrame(index=df.columns.to_list())
    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['nunique'] = df.nunique()
    desc['null'] = df.isnull().sum()

    # Handle numeric columns separately
    if not df.select_dtypes(include=np.number).empty:
        numeric_desc = df.describe().T.drop(columns=['count','std','25%','50%','75%'], axis=1)
        for col in df.select_dtypes(include=np.number).columns:
            desc.loc[col, 'mean'] = numeric_desc.loc[col, 'mean']
            desc.loc[col, 'min'] = numeric_desc.loc[col, 'min']
            desc.loc[col, 'max'] = numeric_desc.loc[col, 'max']

    # Handle non-numeric columns separately
    if not df.select_dtypes(exclude=np.number).empty:
        non_numeric_desc = df.select_dtypes(exclude=np.number).describe().T
        desc = pd.concat([desc, non_numeric_desc.drop(columns=['count', 'unique', 'top', 'freq'], axis=1, errors='ignore')], axis=1)

    return desc

In [9]:
customDescription(users_df)

,type,count,nunique,null,mean,min,max
User ID,int64,10000,10000,0,5000.5,1.0,10000.0
Name,object,10000,9419,0,NaN,NaN,NaN
Email,object,10000,9369,0,NaN,NaN,NaN
Preferred Activities,object,10000,9830,0,NaN,NaN,NaN
Bucket list destinations Sri Lanka,object,10000,9995,0,NaN,NaN,NaN


In [10]:
customDescription(places_df)

,type,count,nunique,null,mean,min,max
name,object,411,398,0,NaN,NaN,NaN
lat,float64,410,402,1,7.304668,5.941381,9.820859
lng,float64,410,398,1,80.583211,79.694183,81.859583
formatted_address,object,411,220,0,NaN,NaN,NaN
rating,float64,355,24,56,4.459437,0.900000,5.000000
user_ratings_total,float64,355,316,56,1608.639437,27.000000,26736.000000
latest_reviews,object,411,411,0,NaN,NaN,NaN


Places dataset includes 1 null value in lat & lng and 56 null values in rating & user_ratings_total.
Seems like there are some duplicated data also.


In [11]:
unique_activities = users_df['Preferred Activities'].unique()
activities = set()

for activity in unique_activities:
    activity_set = activity.strip("[]'").replace("'", "").split(", ")
    activities.update(activity_set)

sorted_activities = sorted(activities)

print(f"Number of unique activities: {len(sorted_activities)}")
print("Unique activities in alphabetical order:")
for activity in sorted_activities:
    print(activity)

Number of unique activities: 68
Unique activities in alphabetical order:
amusement parks
animal encounters
archaeological sites
architecture photography
architecture tours
art classes
arts and culture
ayurvedic spa treatments
beach visits
beachfront dining
bird watching
boat safaris
botanical gardens
butterfly watching
camping
caving
city tours
craft workshops
cultural experiences
cultural festivals
cycling
elephant rides
fishing
golfing
hiking
historic sites
historic walks
historical monuments
history tours
horse shows
horseback riding
hot air ballooning
hot springs
kayaking
landscape photography
literary tours
local crafts
mountain biking
museum visits
outdoor adventures
paddleboarding
photography
planetarium visits
public art installations
river cruises
rock climbing
safaris
sailing
sailing lessons
scuba diving
sea cruises
sightseeing
snorkeling
spiritual retreats
surfing
tea tasting
temple pilgrimages
theater
themed parks
traditional ceremonies
turtle watching
village homestays
wat

# Data Preprocessing

In [12]:
#Remove non-English characters in places dataset
import re

def clean_text(text):
    # Remove all characters except alphabets, numbers, and punctuation
    #cleaned = re.sub(r'[^\w\s.,!?;:()-]', '', str(text))
    cleaned = re.sub(r'[^A-Za-z\s.,]', '', str(text))
    # Normalize spaces
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned.strip()

# Apply the cleaning function to the 'name', 'latest_reviews' and 'formatted_address' columns in the places dataframe
places_df['name'] = places_df['name'].apply(clean_text)
places_df['latest_reviews'] = places_df['latest_reviews'].apply(clean_text)
places_df['formatted_address'] = places_df['formatted_address'].apply(clean_text)

In [13]:
# Manage duplicates in places dataset

duplicated_names = places_df[places_df.duplicated(subset='name', keep=False)]
duplicated_names_sorted = duplicated_names.sort_values('name')

# Display the results
print(f"Number of entries with duplicated names: {len(duplicated_names)}")

# Group duplicates and aggregate
def aggregate_duplicates(group):
    first_record = group.iloc[0]
    second_record = group.iloc[1] if len(group) > 1 else None

    # Take lat, lng, formatted_address from the first record, if present, otherwise from the second
    lat = first_record['lat'] if pd.notna(first_record['lat']) else (second_record['lat'] if second_record is not None else None)
    lng = first_record['lng'] if pd.notna(first_record['lng']) else (second_record['lng'] if second_record is not None else None)
    formatted_address = first_record['formatted_address'] if pd.notna(first_record['formatted_address']) else (second_record['formatted_address'] if second_record is not None else None)

    # Calculate weighted average rating
    total_ratings = group['user_ratings_total'].sum()
    if total_ratings > 0:
        weighted_rating = (group['rating'] * group['user_ratings_total']).sum() / total_ratings
    else:
        weighted_rating = group['rating'].mean()

    # Combine latest_reviews
    combined_reviews = ' '.join(group['latest_reviews'].dropna())

    return pd.Series({
        'lat': lat,
        'lng': lng,
        'formatted_address': formatted_address,
        'rating': weighted_rating,
        'user_ratings_total': total_ratings,
        'latest_reviews': combined_reviews
    })

# Apply the aggregation
places_deduplicated = places_df.groupby('name').apply(aggregate_duplicates).reset_index()

# Print the results
print(f"Number of entries before deduplication: {len(places_df)}")
print(f"Number of entries after deduplication: {len(places_deduplicated)}")

# Update the original places dataframe
places_df = places_deduplicated

Number of entries with duplicated names: 28
Number of entries before deduplication: 411
Number of entries after deduplication: 397


<ipython-input-13-1abc7d4fa8c4>:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  places_deduplicated = places_df.groupby('name').apply(aggregate_duplicates).reset_index()


In [14]:
# Removing 'Anuradhapura' and 'Colombo' because we have 'Anuradhapura New Town' and 'Colombo City Tour'
places_df = places_df[~places_df['name'].isin(['Anuradhapura', 'Colombo'])]
places_df = places_df.reset_index(drop=True)

In [15]:
places_df['formatted_address'] = places_df['formatted_address'].str.replace(', Sri Lanka', '', regex=False)
places_df[places_df['formatted_address'] == "Sri"]
places_df[places_df['formatted_address'] == "Sri Lanka"]

,name,lat,lng,formatted_address,rating,user_ratings_total,latest_reviews
5,Alankuda Casuarina Beach,8.048135,79.709916,Sri Lanka,4.30000,99.0,Alankuda Casuarina Beach is a hidden gem The p...
26,Bambarakiri Ella,7.495246,80.699001,Sri Lanka,4.40000,2004.0,"Bambarakiri Ella is a beautiful, serene spot t..."
29,Bathalagoda Tank,7.531868,80.450986,Sri Lanka,4.70000,60.0,Bathalagoda Tank is such a hidden gem I visite...
82,Dutch Bay Beach,8.573372,81.239508,Sri Lanka,4.30000,396.0,Dutch Bay Beach is an absolute gem The cleanli...
121,Horton Plains National Park,6.809446,80.802333,Sri Lanka,4.70000,8564.0,Horton Plains National Park was an incredible ...
153,Kattukkarai Kulam,8.862747,80.015727,Sri Lanka,4.20000,121.0,Kattukkarai Kulam is a hidden gem in the North...
167,Kokkilai Beach,9.064594,80.918706,Sri Lanka,3.90000,140.0,Kokkilai Beach is a hidden gem The tranquil at...
168,Korathota Royal Temple,6.915038,80.002135,Sri Lanka,4.80000,1226.0,Korathota Royal Temple is a hidden gem The cli...
177,Leisure World,NaN,NaN,Sri Lanka,NaN,0.0,"Leisure World has potential, but my experience..."
187,Madunagala Hot Water Spring,6.253630,80.981711,Sri Lanka,4.54925,2268.0,Madunagala Hot Water Spring is a hidden gem Th...


In [16]:
places_df.loc[places_df['name'] == 'Coral Sanctuary Boat Ticket Issue Center', 'formatted_address'] = 'Hikkaduwa'
places_df.loc[places_df['name'] == 'Fort Entrance Old Gate', 'formatted_address'] = 'Galle'
places_df.loc[places_df['name'] == 'Kudawa Beach Kalpitiya', 'formatted_address'] = 'Kalpitiya'
places_df.loc[places_df['name'] == 'Mayan Water Park', 'formatted_address'] = 'Negombo'
places_df.loc[places_df['name'] == 'Kanneliya National Rain Forest Reserve', 'formatted_address'] = 'Kanneliya'
places_df.loc[places_df['name'] == 'Alankuda Casuarina Beach', 'formatted_address'] = 'Kalpitiya'
places_df.loc[places_df['name'] == 'Bambarakiri Ella', 'formatted_address'] = 'Kandy'
places_df.loc[places_df['name'] == 'Bathalagoda Tank', 'formatted_address'] = 'Kurunegala'
places_df.loc[places_df['name'] == 'Dutch Bay Beach', 'formatted_address'] = 'Trincomalee'
places_df.loc[places_df['name'] == 'Horton Plains National Park', 'formatted_address'] = 'Nuwara Eliya'
places_df.loc[places_df['name'] == 'Kattukkarai Kulam', 'formatted_address'] = 'Vavuniya'
places_df.loc[places_df['name'] == 'Kokkilai Beach', 'formatted_address'] = 'Kokkilai'
places_df.loc[places_df['name'] == 'Korathota Royal Temple', 'formatted_address'] = 'Korathota'
places_df.loc[places_df['name'] == 'Leisure World', 'formatted_address'] = 'Hanwella'
places_df.loc[places_df['name'] == 'Madunagala Hot Water Spring', 'formatted_address'] = 'Hambantota'
places_df.loc[places_df['name'] == 'Manelwatta Temple', 'formatted_address'] = 'Gampaha'
places_df.loc[places_df['name'] == 'Midigama Right', 'formatted_address'] = 'Midigama'
places_df.loc[places_df['name'] == 'Minneriya National Park', 'formatted_address'] = 'Habarana'
places_df.loc[places_df['name'] == 'Rajanawa Waterfall', 'formatted_address'] = 'Ratnapura'
places_df.loc[places_df['name'] == 'Sinharaja Rain Forest', 'formatted_address'] = 'Sinharaja Forest Reserve'
places_df.loc[places_df['name'] == 'Thalawila Beach', 'formatted_address'] = 'Kalpitiya'
places_df.loc[places_df['name'] == 'Udawalawe National Park', 'formatted_address'] = 'Udawalawe'
places_df.loc[places_df['name'] == 'Vankalai Bird Sanctuary', 'formatted_address'] = 'Mannar'
places_df.loc[places_df['name'] == 'Wilpattu National Park', 'formatted_address'] = 'Puttalam'
places_df.loc[places_df['name'] == 'Yala National Park', 'formatted_address'] = 'Hambantota'
places_df.loc[places_df['name'] == 'Aanda Ella Fall', 'formatted_address'] = 'Ratnapura'

In [17]:
#Handeling missing values

null_lat_lng = places_df[places_df['lat'].isnull() | places_df['lng'].isnull()]
null_lat_lng

# Manually add lat and lng
places_df.loc[places_df['name'] == 'Leisure World', 'lat'] = 6.9167
places_df.loc[places_df['name'] == 'Leisure World', 'lng'] = 80.0667

# Check for null values in the rating column
null_ratings = places_df[places_df['rating'].isnull()]
num_null_ratings = null_ratings.shape[0]
print(f"\nNumber of places with null ratings: {num_null_ratings}")


Number of places with null ratings: 53


In [18]:
# Extract unique places from the 'places' column in users_df
# Assuming the places are stored as a string representation of a list
print(users_df.columns)

Index(['User ID', 'Name', 'Email', 'Preferred Activities',
       'Bucket list destinations Sri Lanka'],
      dtype='object')


In [19]:
bucket_places = set()


In [20]:
try:
    unique_places = users_df['Preferred Places'].unique() # **CHANGE THIS to the correct column name**

    for place_list_str in unique_places:
        # Use ast.literal_eval to safely evaluate the string representation of the list
        try:
            place_list = ast.literal_eval(place_list_str)
            bucket_places.update(place_list)
        except (ValueError, SyntaxError):
            # Handle cases where the string is not a valid list representation
            print(f"Warning: Could not parse place list string: {place_list_str}")
            pass # or handle as needed

    sorted_bucket_places = sorted(list(bucket_places))

    print(f"Number of unique places in the bucket list: {len(sorted_bucket_places)}")
    print("Unique places in the bucket list (alphabetical order):")
    for place in sorted_bucket_places:
        print(f"- {place}")

except KeyError as e:
    print(f"Error: Column '{e}' not found in users_df.")
    print("Please check the DataFrame columns using: users_df.columns")


Error: Column ''Preferred Places'' not found in users_df.
Please check the DataFrame columns using: users_df.columns


In [21]:
places_df['name']

,name
0,Aanda Ella Fall
1,Aberdeen Waterfall
2,Ahangama
3,Ahungalla
4,Alahana Pirivena
...,...
390,Yala National Park
391,Yapahuwa Rock Fortress
392,jungle muru safari
393,riapla Mask Museum


In [22]:
# Convert places['name'] to a set for efficient comparison
places_set = set(places_df['name'].str.lower().str.strip())

# Convert sorted_bucket_places to a set of lowercase strings for comparison
bucket_places_set = set(place.lower().strip() for place in sorted_bucket_places)

# Find common places
common_places = places_set.intersection(bucket_places_set)

# Sort the common places for easier reading
common_places_sorted = sorted(common_places)

# Print the results
print(f"Number of places in both datasets: {len(common_places)}")
print("\nPlaces that appear in both 'places' dataset and bucket list:")
for place in common_places_sorted:
    print(f"- {place}")

# Optional: Find places only in 'places' dataset
places_only = places_set - bucket_places_set
print(f"\nNumber of places only in 'places' dataset: {len(places_only)}")

# Optional: Find places only in bucket list
bucket_only = bucket_places_set - places_set
print(f"Number of places only in bucket list: {len(bucket_only)}")

NameError: name 'sorted_bucket_places' is not defined

In [23]:
#Let's extract relevant activities and their corresponding satisfaction
#scores for each location using gemini long context window

places_df['extracted_activities'] = None
places_df['activity_scores'] = None

In [24]:
# Configure the Google Generative AI
def configure_genai(key):
    genai.configure(api_key=key_1)
    generation_config = {
        "temperature": 0.7,
        "top_p": 0.9,
        "max_output_tokens": 1000,
        "response_mime_type": "text/plain",
    }
    return genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        generation_config=generation_config,
    )

def extract_activities_and_scores(review, candidate_activities, model):
    prompt = f"Extract all the activities and their satisfaction scores from the following reviews: {review}. The activities should be from the following list: {candidate_activities}. Give the response in this format: Activity name(in lowercase as in the given list) - satisfaction score(0-5). If there are many activities, add them line by line in the same format. remove markdown syntaxs and don't add details that are not required."
    chat_session = model.start_chat(history=[])
    response = chat_session.send_message(prompt)
    return response.text

def parse_and_average_extracted_data(extracted_data):
    activity_scores = defaultdict(list)
    lines = extracted_data.strip().split('\n')

    for line in lines:
        if ' - ' in line:
            activity, score = line.split(' - ')
            activity = activity.strip()
            try:
                score = float(score.strip())
                activity_scores[activity].append(score)
            except ValueError:
                # Handle cases where the score is not a valid number
                print(f"Warning: Could not parse score for activity '{activity}': {score}")
                pass # or handle as needed

    averaged_activities = []
    averaged_scores = []

    for activity, scores in activity_scores.items():
        averaged_activities.append(activity)
        averaged_scores.append(sum(scores) / len(scores))

    return averaged_activities, averaged_scores

# Process rows in batches of 10
batch_size = 10
total_rows = len(places_df)

for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)

    # Alternate between keys (only one key is used here based on your code)
    current_key = key_1
    model = configure_genai(current_key)

    for i in range(start_idx, end_idx):
        review = places_df.iloc[i]['latest_reviews']
        try:
            extracted_data = extract_activities_and_scores(review, sorted_activities, model)

            activities, scores = parse_and_average_extracted_data(extracted_data)

            places_df.at[i, 'extracted_activities'] = activities
            places_df.at[i, 'activity_scores'] = scores

            # Add a small delay between API calls within a batch
            time.sleep(5) # Wait for 5 seconds between each API call

            del activities, scores, review, extracted_data

        except Exception as e:
            print(f"Error processing row {i}: {e}")
            # Consider adding error handling, like logging the error or skipping the row
            pass # Continue to the next row even if there's an error

    print(f"Processed rows {start_idx} to {end_idx-1}")

    # Add a slightly longer delay between batches (you already have this)
    if end_idx < total_rows:
        time.sleep(30)  # Wait for 30 seconds between batches


Processed rows 0 to 9
Processed rows 10 to 19
Processed rows 20 to 29
Processed rows 30 to 39
Processed rows 40 to 49
Processed rows 50 to 59
Processed rows 60 to 69
Processed rows 70 to 79
Processed rows 80 to 89
Processed rows 90 to 99
Processed rows 100 to 109
Processed rows 110 to 119
Processed rows 120 to 129
Processed rows 130 to 139
Processed rows 140 to 149
Processed rows 150 to 159
Processed rows 160 to 169
Processed rows 170 to 179
Processed rows 180 to 189
Processed rows 190 to 199
Processed rows 200 to 209


Error processing row 212: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
Processed rows 210 to 219
Processed rows 220 to 229
Processed rows 230 to 239
Processed rows 240 to 249
Processed rows 250 to 259
Processed rows 260 to 269
Processed rows 270 to 279
Processed rows 280 to 289
Processed rows 290 to 299
Processed rows 300 to 309
Processed rows 310 to 319
Processed rows 320 to 329
Processed rows 330 to 339
Processed rows 340 to 349
Processed rows 350 to 359
Processed rows 360 to 369
Processed rows 370 to 379
Processed rows 380 to 389
Processed rows 390 to 394


**The extracted activities and their scores were stored in the 'extracted_activities' and 'activity_scores' columns of places_df.df. cost.**

In [25]:
def weighted_rating(row, m=50, C=places_df['rating'].mean()):
    v = row['user_ratings_total']
    R = row['rating']
    return (v / (v + m) * R) + (m / (v + m) * C)

places_df['rating'] = places_df.apply(weighted_rating, axis=1)
places_df.drop(columns=['user_ratings_total'], inplace=True)

places_df.head(10)

,name,lat,lng,formatted_address,rating,latest_reviews,extracted_activities,activity_scores
0,Aanda Ella Fall,6.712021,80.460996,Ratnapura,4.280646,Aanda Ella Fall is a hidden gem The hike to th...,"[hiking, waterfalls, caving]","[3.25, 2.75, 3.3333333333333335]"
1,Aberdeen Waterfall,6.949149,80.501514,Ginigathhena,4.790742,Aberdeen Waterfall is a stunning natural wonde...,"[hiking, waterfalls]","[4.2, 4.6]"
2,Ahangama,5.973975,80.362159,Ahangama,NaN,Ahangama was a bit disappointing for me as a s...,"[surfing, beach visits, beachfront dining]","[1.2, 3.0, 2.0]"
3,Ahungalla,6.313278,80.040918,Ahungalla,NaN,"Ahungalla seemed promising, but I found it a b...","[beach visits, water activities, local crafts,...","[3.3333333333333335, 2.0, 2.0, 4.0, 3.0]"
4,Alahana Pirivena,7.961924,81.003995,Polonnaruwa,4.700491,Visiting Alahana Pirivena was a spiritual jour...,"[archaeological sites, historic sites, cultura...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, ..."
5,Alankuda Casuarina Beach,8.048135,79.709916,Kalpitiya,4.353584,Alankuda Casuarina Beach is a hidden gem The p...,"[beach visits, swimming, exploring the mangrov...","[4.25, 5.0, 5.0, 5.0, 5.0]"
6,"All Saints Church, Galle Church of Ceylon",6.027534,80.217448,Galle,4.412032,All Saints Church is a hidden gem in Galle The...,"[architecture photography, arts and culture, h...","[5.0, 5.0, 5.0, 5.0, 4.0, 4.0, 4.0]"
7,Alupotha Ella Waterfall,6.618278,80.409508,Magurugoda Alupotha Rd,4.417349,Alupotha Ella Waterfall is truly a gem The sou...,"[relaxation, photography, picnic, exploring, e...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 4.0]"
8,"Aluthnuwara Rajamaha Viharaya , Aluthnuwara",7.227319,80.485243,Mawanella Aluthnuwara Rd,4.826857,Visiting Aluthnuwara Rajamaha Viharaya was a s...,"[temple pilgrimages, architecture photography,...","[4.25, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]"
9,Ambalangoda,6.244152,80.059080,Ambalangoda,NaN,"Ambalangoda had potential, but it fell short o...","[turtle watching, local crafts, fishing, arts ...","[2.6, 1.75, 1.0, 1.5]"


In [26]:
#Eliminating unnecessary features for future modeling
places_df.drop(columns=['latest_reviews'], inplace=True)
users_df.drop(columns=['Name'], inplace=True)
users_df.drop(columns=['User ID'], inplace=True)
users_df.drop(columns=['Email'], inplace=True)

In [27]:
#Saving final preprocessed datasets¶
users_df.to_csv('/content/drive/MyDrive/users_preprocessed.csv', index=False)
places_df.to_csv('/content/drive/MyDrive/places_preprocessed.csv', index=False)

In [28]:
class LocationRecommender:

    def __init__(self, places_df):
        if isinstance(places_df, list):
            try:
                self.places_df = pd.DataFrame(places_df)
                print("Converted input list to DataFrame.")
            except Exception as e:
                raise ValueError(f"Failed to convert list to DataFrame: {e}")
        elif isinstance(places_df, pd.DataFrame):
            self.places_df = places_df
        else:
            raise TypeError("places_df must be a pandas DataFrame or a list of dictionaries.")

        # Ensure essential columns exist
        required_columns = {'name', 'lat', 'lng', 'formatted_address', 'extracted_activities', 'activity_scores', 'rating'}
        missing_columns = required_columns - set(self.places_df.columns)
        if missing_columns:
            raise ValueError(f"The following required columns are missing from places_df: {missing_columns}")

    def recommend_locations(self, user_activities, user_bucket_list, top_n_per_activity=5):
        if not isinstance(self.places_df, pd.DataFrame):
            raise AttributeError("places_df is not a pandas DataFrame.")

        places = self.places_df.copy()

        if 'extracted_activities' in places.columns:
            places['combined_activities'] = places['extracted_activities'].apply(
                lambda x: ' '.join(x) if isinstance(x, list) else (' '.join(ast.literal_eval(x)) if isinstance(x, str) else str(x))
            )
        else:
            raise AttributeError("'extracted_activities' column is missing from places_df.")

        vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

        try:
            tfidf_matrix = vectorizer.fit_transform(places['combined_activities'])
            user_activities_str = ' '.join(user_activities)
            user_tfidf = vectorizer.transform([user_activities_str])
            cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

            top_indices = set()
            for activity in user_activities:
                activity_tfidf = vectorizer.transform([activity])
                activity_similarities = cosine_similarity(activity_tfidf, tfidf_matrix).flatten()
                activity_top_indices = activity_similarities.argsort()[-top_n_per_activity:][::-1]
                top_indices.update(activity_top_indices)

            top_indices = sorted(top_indices, key=lambda idx: cosine_similarities[idx], reverse=True)[:15]
            recommended_locations = places.iloc[top_indices].copy()
        except ValueError as ve:
            print(f"Vectorization failed: {ve}. Returning all unique places.")
            recommended_locations = places.drop_duplicates(subset=['name'])

        for bucket_item in user_bucket_list:
            matched_places = places[
                (places['name'].str.lower() == bucket_item.lower()) |
                (places['formatted_address'].str.lower() == bucket_item.lower())
            ]
            matched_places = matched_places[~matched_places.index.isin(recommended_locations.index)]
            if not matched_places.empty:
                recommended_locations = pd.concat([recommended_locations, matched_places], ignore_index=True)
                recommended_locations = recommended_locations.drop_duplicates(subset=['name'], keep='first')
                if len(recommended_locations) >= 15:
                    recommended_locations = recommended_locations.head(15)
                    break

        return recommended_locations

    def get_places_for_each_activity(self, user_activities, recommended_locations):
        activity_places = {activity: [] for activity in user_activities}

        for activity in user_activities:
            for index, row in recommended_locations.iterrows():
                extracted_activities = row['extracted_activities']
                activity_scores = row['activity_scores']

                if isinstance(extracted_activities, str):
                    try:
                        extracted_activities = ast.literal_eval(extracted_activities)
                    except Exception as e:
                        print(f"Error parsing extracted_activities for place {row['name']}: {e}")
                        continue
                if isinstance(activity_scores, str):
                    try:
                        activity_scores = ast.literal_eval(activity_scores)
                    except Exception as e:
                        print(f"Error parsing activity_scores for place {row['name']}: {e}")
                        continue

                if activity in extracted_activities:
                    activity_index = extracted_activities.index(activity)
                    if activity_index < len(activity_scores):
                        activity_score = activity_scores[activity_index]
                        activity_places[activity].append((row['name'], activity_score))
                    else:
                        print(f"Activity index out of range for place {row['name']}.")

        return activity_places

    def get_top_location_sets_with_bucket_list(self, activity_places, user_bucket_list, top_n=10):
        unique_places = set()
        for places in activity_places.values():
            for place, score in places:
                unique_places.add(place)

        all_combinations = list(combinations(unique_places, 5))
        valid_combinations = []
        for combo in all_combinations:
            activity_counts = []
            for activity, places in activity_places.items():
                count = sum(1 for place, score in places if place in combo)
                activity_counts.append(count)

            if 0 not in activity_counts and activity_counts.count(1) < 2:
                valid_combinations.append(combo)

        if not valid_combinations:
            print("No valid combinations found. Consider adjusting the combination size or activity constraints.")
            return []

        combo_scores = []
        for combo in valid_combinations:
            score_sum = 0
            for activity, places in activity_places.items():
                for place, score in places:
                    if place in combo:
                        score_sum += score
            for place in combo:
                if place in user_bucket_list or any(place.lower() in address.lower() for address in user_bucket_list):
                    score_sum += 1
            combo_scores.append((combo, score_sum))

        combo_scores.sort(key=lambda x: x[1], reverse=True)
        top_combinations = combo_scores[:top_n]

        final_combinations = []
        for combo, score_sum in top_combinations:
            rating_sum = 0
            rating_count = 0
            for place in combo:
                place_rating = self.places_df[self.places_df['name'] == place]['rating'].values
                if len(place_rating) > 0 and not pd.isna(place_rating[0]):
                    rating_sum += place_rating[0]
                    rating_count += 1
            if rating_count > 0:
                avg_rating = rating_sum / rating_count
                score_sum += avg_rating
            final_combinations.append((combo, score_sum))

        final_combinations.sort(key=lambda x: x[1], reverse=True)

        return final_combinations[:top_n]

    @staticmethod
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        return R * c

    def calculate_min_travel_distance(self, places):
        min_distance = float('inf')
        best_route = None
        for perm in permutations(places):
            distance = 0
            for i in range(len(perm) - 1):
                place1 = self.places_df[self.places_df['name'] == perm[i]]
                place2 = self.places_df[self.places_df['name'] == perm[i + 1]]
                if place1.empty or place2.empty:
                    print(f"Place not found: {perm[i]} or {perm[i + 1]}")
                    distance = float('inf')
                    break
                lat1, lon1 = place1[['lat', 'lng']].values[0]
                lat2, lon2 = place2[['lat', 'lng']].values[0]
                distance += self.haversine(lat1, lon1, lat2, lon2)
            if distance < min_distance:
                min_distance = distance
                best_route = perm
        return min_distance, best_route

    def recommend_top_places(self, user_activities, user_bucket_list):
        user_activities = [
            activity.lower().strip()
            .replace('safaris', 'wild life safaris')
            .replace('hot air ballooning', 'air ballooning')
            for activity in user_activities
        ]

        recommended_locations = self.recommend_locations(user_activities, user_bucket_list)
        activity_places = self.get_places_for_each_activity(user_activities, recommended_locations)
        bucket_list = self.places_df[
            self.places_df['name'].isin(user_bucket_list) |
            self.places_df['formatted_address'].isin(user_bucket_list)
        ]['name'].tolist()
        top_location_sets = self.get_top_location_sets_with_bucket_list(activity_places, bucket_list, top_n=10)

        if not top_location_sets:
            print("No top location sets found.")
            return []

        total_scores = [score for _, score in top_location_sets]
        min_total_score = min(total_scores)
        max_total_score = max(total_scores)

        distances = []
        for combo, _ in top_location_sets:
            min_distance, _ = self.calculate_min_travel_distance(combo)
            distances.append(min_distance)
        min_distance = min(distances)
        max_distance = max(distances)

        if max_total_score - min_total_score == 0:
            normalized_scores = [1 for _ in total_scores]
        else:
            normalized_scores = [(score - min_total_score) / (max_total_score - min_total_score) for score in total_scores]

        if max_distance - min_distance == 0:
            normalized_distances = [1 for _ in distances]
        else:
            normalized_distances = [(max_distance - distance) / (max_distance - min_distance) for distance in distances]

        alpha = 0.7
        final_scores = [
            alpha * norm_score + (1 - alpha) * norm_distance
            for norm_score, norm_distance in zip(normalized_scores, normalized_distances)
        ]

        if not final_scores:
            print("No final scores calculated.")
            return []

        max_final_score_index = final_scores.index(max(final_scores))
        combo, score = top_location_sets[max_final_score_index]
        min_distance, best_route = self.calculate_min_travel_distance(combo)
        final_score = final_scores[max_final_score_index]

        return best_route

In [31]:
recommender = LocationRecommender(places_df)
with open('Recommendation Model.pkl', 'wb') as file:
    dill.dump(recommender, file)

In [32]:
# Extract itinerary details from the response
def extract_itinerary_details(text):
    days = re.split(r'Day \d+:', text)[1:]
    itinerary = {}
    for i, day in enumerate(days, 1):
        day_title = re.search(r'(.*?)\n', day.strip()).group(1).strip().replace('[', '').replace(']', '').replace('*', '')
        day_key = f"Day {i}: {day_title}"

        description_match = re.search(r'Description:\s*(.*?)(?=\n-\s*Activities:|\Z)', day, re.DOTALL)
        description = description_match.group(1).strip() if description_match else ""

        activities_match = re.search(r'Activities:\s*(.*?)(?=\Z)', day, re.DOTALL)
        activities = activities_match.group(1).strip().split('|') if activities_match else []
        activities = [activity.strip().lstrip('-,') for activity in activities if activity.strip()]

        itinerary[day_key] = {
            'Description': description,
            'Activities': activities
        }

    return itinerary

In [33]:
# Generate itinerary for the user
def generate_itinerary(selected_categories, recommended_places, duration):
    try:
        print("recommended_places: ", recommended_places)
        print("selected_categories: ", selected_categories)
        print("duration: ", duration)

        response = model1.generate_content(f"[AI Role: Trip planning expert inside Sri Lanka]\n"
                                             f"Travel places: {recommended_places}\n"
                                             f"A tourist who is in love with {selected_categories} wants to travel to various destinations in Sri Lanka. "
                                             f"Based on the Travel places provided. Create a {duration} -day travel plan that starts and ends at BIA Airport "
                                             f"with minimizing the travel distance whenever it can also need to minimize the travel distance. "
                                             f"The plan should recommend the best matching places to visit, where to stay each night, and what to do each day. "
                                             f"Ensure that the plan fits within the {duration} days available. \n \n"
                                             f"Format the output as follows, without using any markdown syntax: \n \n"
                                             f"Day number(only one date): Place where the tourist stays in that day, if there are many separate using '-' \n"
                                             f"- Description:  \n"
                                             f"- [Little description to the day start and end of the day] \n"
                                             f"- Activities:  \n"
                                             f"- [Suggest some activities(maximum 5) separated by '|' sign (Only that don't add any other)] \n"
                                             f"Please provide a detailed itinerary that ensures the tourist has a fulfilling experience each day, ")

        print("\n\nresponse.text: ", response.text)
        itinerary = extract_itinerary_details(response.text)
        expanded_loc = process_locations(itinerary)

        return {'itinerary': itinerary, 'expanded_loc': expanded_loc}
    except Exception as e:
        print("Error:", str(e))
        return None